In [ ]:
# Copyright (C) 2025  Scott Lebow and Krisztian Hajdu

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# Author contact:
# Scott Lebow: scott.lebow@student.iaac.net
# Krisztian Hajdu: krisztian.hajdu@students.iaac.net

In [22]:
# Set the working directory to the next level up
import os
import pandas as pd
script_dir = os.getcwd()

if script_dir.endswith('\\notebooks'):
    new_dir = '\\'.join(script_dir.split('\\')[:-1])
else:
    new_dir = script_dir
os.chdir(new_dir)
print(f"Current working directory: {new_dir}")

# Load the unique requirements csv file to verify the data
unique_df = pd.read_excel('tables/unique_requirements.xlsx')

Current working directory: c:\Users\scleb\Documents\bimsc25\GitHub\MACAD-Thesis-MEP-Graph


In [23]:
columns = unique_df.columns.tolist()
print(f"Columns in unique_requirements.csv: {columns}")

Columns in unique_requirements.csv: ['Attribute Name', 'Description', 'Unit Type', 'Unit Description', 'Example Value 1', 'Example Value 2', 'Required for']


In [24]:
# Change 'Required for' column to a list of strings
unique_df['Required for (list)'] = unique_df['Required for'].apply(lambda x: [item.strip() for item in x.split('\n') if item.strip()])


In [25]:
component_lists = unique_df['Required for (list)'].tolist()

all_components = set()

for component_list in component_lists:
    for component in component_list:
        all_components.add(component)

component_types = list(all_components)
print(f"Unique component types: {len(component_types)}")
for component in component_types:
    print(component)

Unique component types: 58
Power Supply - Lighting - Self-contained emergency lighting systems - Emergency luminaires
Power Supply - Cable arrangement - High-voltage systems - Power busbar trunking
Power Supply - Grounding and lightning protection - Lightning protection systems - Lightning protection and earthing components
Power Supply - Lighting - General lighting systems - Infrared lamps
Power Supply - Power transmission - Low-voltage systems - Electrical protective devices
Power Supply - Cable arrangement - High-voltage systems - Cable ladders
Power Supply - Power transmission - High-voltage systems - Distribution boards
Power Supply - Power transmission - Low-voltage distribution systems - Service outlet poles
Power Supply - Power transmission - Low-voltage distribution systems - Standard socket outlets
Power Supply - Power transmission - High-voltage systems - Electrical protective devices
Power Supply - Lighting - External lighting systems - Area floodlights
Power Supply - Light

In [26]:
# Create a new dataframe with the unique component types
component_types_df = pd.DataFrame(component_types, columns=['Component Type'])

# Create a new column in the component_types_df for each 'Attribute Name' in the unique_df
unique_attributes = unique_df['Attribute Name'].unique()
component_types_df = component_types_df.reindex(columns=component_types_df.columns.tolist() + list(unique_attributes))

# Fill the new columns with True/False based on whether the 'Component Type' is in the 'Required for (list)' of the unique_df for each 'Attribute Name'
for attribute in unique_attributes:
    component_types_df[attribute] = component_types_df['Component Type'].apply(
        lambda x: x in unique_df[unique_df['Attribute Name'] == attribute]['Required for (list)'].explode().tolist()
    )

print("Component types dataframe created with unique attributes:")
print(component_types_df.head())

Component types dataframe created with unique attributes:
                                      Component Type  Color rendering index  \
0  Power Supply - Lighting - Self-contained emerg...                   True   
1  Power Supply - Cable arrangement - High-voltag...                  False   
2  Power Supply - Grounding and lightning protect...                  False   
3  Power Supply - Lighting - General lighting sys...                   True   
4  Power Supply - Power transmission - Low-voltag...                  False   

   Color temperature  Connector types  Contractor  Cross-section  End 1  \
0               True            False       False          False  False   
1              False            False       False          False  False   
2              False            False       False          False  False   
3               True            False       False          False  False   
4              False            False       False          False  False   

   End 2  Functi

In [27]:
# Save the filtered DataFrame to a new csv file
component_types_df.to_csv('tables/component_types.csv', index=False)